# Otro Ejemplo

Este ejemplo se dearrolló en el lenguaje `Julia` como una implementación del algoritmo antes mencionado. Para ello se construyeron datos que estuvieran en grupos aislados, posteriormente se realiza un tratamiento que nos permitió extraer una muestra adaptativa por clouster.

1. En la primera parte de este ejemplo se contruyeron las funciones necesarias para este método
    - la función `vecindad` que nos da los vecinos de un elemento seleccionado en una matriz, se considera periosidad en la frontera para facilidar los cálculos
    - la función `crece` guarda las unidades que puede crecer una unidad incial, hasta tener unidades de borde.
    - finalmente la función `redes`  nos da las $m_i$ y $y_i$ de $i$ unidades iniciales      

2. En la segunda parte del ejemplo, visualizamos los datos y contamos cuántos datos hay en cada unidad, este conteo lo realizamos en una matriz, los que nos permite llamar a cada elemento de la matriz como $u_i$

3. Finalmente aplicamos nuestras funciones y visualizamos como las muestras crecieron bajo la condición de la unidad, esta es si $u_i > C$ con $C$ una constante. Agrupamos nuestros resultados en una tabla y calculamos el estimador de la media según lo visto en la explicación.

### Funciones

In [2]:
#Cargamos paqueterias
using Plots, Statistics,CSV, DataFrames
using LinearAlgebra
using ColorSchemes

In [3]:
#dado un  elementos quiero conocer sus vecinos, Matriz y coordenada de la unidad 
function vecindad(M,ui)
	n,m= size(M) 
	vecino(k,a)= mod1(k,a) #vecinos con periocidad

    i= ui[1]
    j= ui[2]

	vecinos= [  M[vecino(i-1,n), j], 
			    M[vecino(i+1,n), j], 
			    M[i, vecino(j-1,m)], 
		        M[i, vecino(j+1,m)]]
	
	coordenadas= [ (vecino(i-1,n), j), 
					(vecino(i+1,n), j), 
					(i, vecino(j-1,m)), 
					(i, vecino(j+1,m))]

	return vecinos, coordenadas
end

vecindad (generic function with 1 method)

In [4]:
# La función que crecerá según un valor C, me arroja coordenas en la matriz

function crece(U, s_i, C)

visitados= Set()
cola= [s_i]
Ci=[s_i]  

while !isempty(cola)
    celda = popfirst!(cola)

    if celda in visitados
        continue
    end

    push!(visitados, celda)

    x,y = celda
    if U[x,y] > C
        push!(Ci, celda)

        vecinos = vecindad(U, celda)
        for i in 1:4
            if vecinos[2][i] ∉ visitados
                push!(cola, vecinos[2][i])
            end
        end
  
    end
end
return unique(Ci)

end



crece (generic function with 1 method)

In [5]:
function redes(U, mas, C)
    l = length(mas)
    mi =  []
    y = []
    ni= []
    

    for i in 1:l
        r= crece(U,mas[i],C)
        n= length(r)
        mi= push!(mi,n)
        yi= [U[r[j][1],r[j][2]] for j in 1:n] 
        y= push!(y,yi)
        ni= push!(ni, r)
    end
    
    return  mi,  y, ni
end

redes (generic function with 1 method)

### Graficamos datos 

In [15]:
#Cargamos datos y graficamos
data = DataFrame(CSV.File("data.csv"))

scatter(data.x,data.y, ratio= 1, color= "purple", label= "", title= "Datos")

savefig("~/Escritorio/especialidad/sampling/proyecto/img/Datosejem3.png")

"/home/quicho/Escritorio/especialidad/sampling/proyecto/img/Datosejem3.png"

Contamos estos datos por unidad en el plano y lo representamos como una matriz $U = \{u_1,u_2, \cdots, u_N\}$

In [7]:
#una matriz para almacenar las cuentas
U = zeros(Int, 20, 20) #unidades

x_i = floor.(Int, data.x)  
y_i = floor.(Int, data.y)  

for (i, j) in zip(x_i, y_i)
    if 1 <= i <= 20 && 1 <= j <= 20
        U[i, j] += 1
    end
end

U

20×20 Matrix{Int64}:
  0  0   0  0   0   0  0  0  0  0  0   0   0   0   0   0   0   0  0  0
 15  8   0  0   0   0  0  0  0  0  0   0   5   5   0   0   0   0  0  0
  0  6   0  0   0   0  0  0  0  0  0   5   5   5   6   5   0   0  0  0
  0  0   0  0   0   0  0  0  0  5  0   5  15  11   0  10   0   0  0  0
  0  0   0  0   0   0  0  0  0  0  0   5  20  15  11  11   0   0  0  0
  0  0   0  0   0   0  0  0  0  0  0  10   5   0  20  14   0   0  0  0
  0  0   0  0   0   0  0  0  0  0  0   0   0   0   0   0   5   0  0  0
  0  0   5  0   0   0  0  0  0  0  0   0   0   0   0   0   0   0  0  0
  0  0   0  0   0   0  0  0  0  0  0   0   0   0   5   0   0   0  0  0
  0  0  10  7  16   6  0  0  0  0  0   0   0   0   0   0   0   0  0  0
  0  0   0  6   2   6  0  0  0  0  0   0   0   2   0   0   0   0  0  0
  0  0  10  8  18  12  2  0  0  0  0   0   0   0   2   0   0   0  0  0
  0  0   4  2   0   0  0  0  0  0  0   0   0   0   0   0   0   0  0  0
  0  0   0  0   0   0  0  0  0  0  0   0   0   0   4  21

Representamos esta matriz como un mapa de calor

In [16]:
#  mapa de calor
custom_colormap = cgrad([:white, :blue, :green, :yellow, :red], [0.0, 0.1, 0.4, 0.7, 1.0])

mapa_calor= heatmap(
    U,
    title = "Datos en cada unidad",
    xlabel = "x",
    ylabel = "y",
    color = custom_colormap,
    clims = (0, maximum(U)), 
    aspect_ratio = 1          
)

savefig("~/Escritorio/especialidad/sampling/proyecto/img/Datosunidadejem3.png")

"/home/quicho/Escritorio/especialidad/sampling/proyecto/img/Datosunidadejem3.png"

**Seleccionamos $i$ unidades con m.a.s**

In [9]:
mas= [(rand(1:20), rand(1:20)) for i in 1:13];  #13 unidades

In [10]:
muestra = redes(U, mas, 3)

(Any[1, 1, 1, 20, 11, 1, 1, 1, 1, 1, 1, 1, 1], Any[[0], [0], [0], [10, 5, 11, 6, 14, 11, 5, 20, 15, 5, 11, 5, 20, 5, 15, 5, 5, 5, 5, 10], [10, 7, 6, 16, 8, 6, 10, 18, 6, 4, 12], [0], [0], [0], [0], [5], [0], [0], [0]], Any[[(11, 15)], [(2, 7)], [(20, 14)], [(4, 16), (3, 16), (5, 16), (3, 15), (6, 16), (5, 15), (3, 14), (6, 15), (5, 14), (2, 14), (4, 14), (3, 13), (5, 13), (2, 13), (4, 13), (3, 12), (6, 13), (5, 12), (4, 12), (6, 12)], [(10, 3), (10, 4), (11, 4), (10, 5), (12, 4), (10, 6), (12, 3), (12, 5), (11, 6), (13, 3), (12, 6)], [(16, 3)], [(18, 11)], [(4, 19)], [(5, 8)], [(8, 3)], [(9, 14)], [(14, 6)], [(19, 2)]])

Visualizamos en el mapa de calor como avanzó cada unidad inicial, notaremos que solo una de ellas tenía vecinos que cumplian la condición de $u_i > C=3$ y se detuvo cuando tenia unidades de borde

In [17]:
function resaltar_zona(zona)
    xs = [x for (x, y) in zona]
    ys = [y for (x, y) in zona]
    scatter!(ys, xs, label= "")
end

mapa_calor

resaltar_zona.(muestra[3])
plot!(title= "Crecimiento de u_i")

savefig("~/Escritorio/especialidad/sampling/proyecto/img/Samplingejem3.png")

"/home/quicho/Escritorio/especialidad/sampling/proyecto/img/Samplingejem3.png"

Finalmente agregamos nuestros resultados a una tabla 

In [12]:
wi= [(1/muestra[1][i]) * sum(muestra[2][i]) for i in 1:13]

resultados= DataFrame(
    m_i = muestra[1],
    y_i = muestra[2],
    w_i = wi 
)

Row,m_i,y_i,w_i
,Any,Any,Float64
1,1,[0],0.0
2,1,[0],0.0
3,1,[0],0.0
4,20,"[10, 5, 11, 6, 14, 11, 5, 20, 15, 5, 11, 5, 20, 5, 15, 5, 5, 5, 5, 10]",9.4
5,11,"[10, 7, 6, 16, 8, 6, 10, 18, 6, 4, 12]",9.36364
6,1,[0],0.0
7,1,[0],0.0
8,1,[0],0.0
9,1,[0],0.0


In [13]:
#guardamos la tabla
CSV.write("resultados_ejemplo3.csv", resultados)

"resultados_ejemplo3.csv"

Con esto podemos estimar la media $\hat{\mu}$ con su respectiva varianza

In [14]:
N= 20*20
n= sum(resultados.m_i) #tamaño de muestra 
media_est= 1/n * sum(resultados.w_i)
varianza_est= (N-n)/(N*n*(n-1)) * sum((resultados.w_i .- media_est).^2)

println("La media estimada= $media_est")
print("Con varianza=$varianza_est")


La media estimada= 0.5658008658008659
Con varianza=0.09267481341125286

Por lo tanto el estimador total de la población sería $N \cdot \hat{\mu} = 0.597 \cdot 400= 238.8 \approx 239$ de datos. En realidad se graficaron 450 datos.